# HSMA 6 - Session 4E - Boosted Trees Exercise 2 - Regression Trees

## Core

We're going to work with a dataset to try to predict patient length of stay. 

In [ ]:
import pandas as pd
import numpy as np

# import preprocessing functions
from sklearn.model_selection import train_test_split

# Import machine learning model of interest
from sklearn.tree import DecisionTreeClassifier, plot_tree, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from xgboost.sklearn import XGBClassifier, XGBRegressor
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.ensemble import HistGradientBoostingClassifier, HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, \
                            r2_score, root_mean_squared_error
# Import package to investigate our loaded dataframe
from ydata_profiling import ProfileReport

# Import functions for evaluating model
from sklearn.metrics import recall_score, precision_score, f1_score, classification_report, \
                            confusion_matrix, ConfusionMatrixDisplay, auc, roc_curve
from sklearn.metrics import auc, roc_curve, RocCurveDisplay, f1_score, precision_score, \
                            recall_score, confusion_matrix, ConfusionMatrixDisplay, \
                            classification_report
from sklearn.inspection import permutation_importance

# Imports relating to logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# Imports relating to plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

Open the data dictionary in the los_dataset folder and take a look at what data is available.

Next, load in the dataframe containing the LOS data.

In [ ]:
los_df = pd.read_csv("../datasets/los_dataset/LengthOfStay.csv", index_col="eid")

View the dataframe.

In [ ]:
los_df.head()

Consider what columns to remove. 

HINT: Is there a column in the dataset that doesn't really make much sense to predict from? 
If you're not sure, use the full dataset for now and come back to this later.

NOTE: For now, we're going to assume that all of the included measures will be available to us at the point we need to make a prediction - they're not things that will be calculated later in the patient's stay.

In [ ]:
data = los_df.copy()
data['date'] = pd.to_datetime(data['vdate'])
data['month'] = data['date'].dt.month
data['month']
data.drop('vdate', axis=1, inplace=True)
data.drop('date', axis=1, inplace=True)

Convert categories with only two options into a boolean value (e.g. for a gender column in which gender has only been provided as M or F, you could encode M as 0 and F as 1).

In [ ]:
data['is_male'] = data['gender'].apply(lambda x: 1 if x == 'M' else 0)
data.drop('gender', axis=1, inplace=True)


Convert columns with multiple options per category into multiple columns using one-hot encoding. 

In [ ]:
def do_one_hot(data, column):
    one_hot = pd.get_dummies(data[column], prefix=column)
    data = data.drop(column, axis=1)
    data = pd.concat([data, one_hot], axis=1)
    return data

columns = ['rcount', 'facid', 'month']

for column in columns:
    data = do_one_hot(data, column)
data.head()

Train a decision tree model to predict length of stay based on the variables in this dataset. 

In [ ]:
X = data.drop('lengthofstay', axis=1)
y = data['lengthofstay'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

def train_and_display(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))
    cm = confusion_matrix(y_test, y_pred, normalize='true')
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    plt.show()
    return f1_score(y_test, y_pred, average='weighted')



Assess the performance of this model.

In [ ]:


def regressor_fit(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    plot_actual_vs_predicted(
    y_test, 
    y_pred
    )

    print(f"Mean absolute error: {mean_absolute_error(y_test, y_pred):.2f}")

    print(f"Mean absolute percentage error: {mean_absolute_percentage_error(y_test, y_pred):.2%}" )

    print(f"Root Mean squared error: {root_mean_squared_error(y_test, y_pred):.2f}")

    print(f"Coefficient of determination: {r2_score(y_test, y_pred):.2f}") 


model = DecisionTreeRegressor()
regressor_fit(model, X_train, y_train, X_test, y_test)

model = XGBRegressor()
regressor_fit(model, X_train, y_train, X_test, y_test)

Train a boosting model to predict length of stay based on the variables in this dataset.

In [ ]:
model = XGBRegressor()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

def plot_actual_vs_predicted(actual, predicted):
    fig, ax = plt.subplots(figsize=(6, 6))

    ax.scatter(actual, predicted, color="black")
    ax.axline((1, 1), slope=1)
    plt.xlabel('True Values')
    plt.ylabel('Predicted Values')
    plt.title('True vs Predicted Values')
    plt.show()


plot_actual_vs_predicted(
y_test, 
y_pred
)

y_test_XGB = y_test
y_pred_XGB = y_pred

model = DecisionTreeRegressor()
model.fit(X_train, y_train)

y_pred_tree = model.predict(X_test)
y_test_tree = y_test


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_actual_vs_predicted(actual, predicted, jitter_amount=0.3):

    actual_jittered = actual + np.random.normal(0, jitter_amount, size=actual.shape)

    fig, ax = plt.subplots(figsize=(6, 6))
    ax.scatter(actual_jittered, predicted, color="blue", alpha=0.3)
    ax.axline((1, 1), slope=1)
    plt.xlabel('True Values')
    plt.ylabel('Predicted Values')
    plt.title('True vs Predicted Values (with Jitter)')
    plt.show()

plot_actual_vs_predicted(y_test, y_pred)

Assess the performance of this model and compare it with your decision tree model. 

In [ ]:

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
ax1.scatter(y_test_XGB, y_pred_XGB, color="blue")
ax1.axline((1, 1), slope=1)
plt.xlabel('True Values')
plt.ylabel('Predicted Values')
plt.title('True vs Predicted Values')

ax2.scatter(y_test_tree, y_pred_tree, color="red")
ax2.axline((1, 1), slope=1)
plt.xlabel('True Values')
plt.ylabel('Predicted Values')
plt.title('True vs Predicted Values')
plt.show()


## Extension

Continue to refine your model. 

You could
- tune hyperparameters
- try additional model types (e.g. different boosting trees, linear regression)
- try dropping some features
- calculate additional metrics
- add some visualisations of the features in the dataset
- store the results from your different model iterations and display this as a table of metrics
- turn this table of metrics into a graph
- try using k-fold cross validation 

In [ ]:
# YOUR CODE HERE

## Challenge

Explore additional aspects of regression modelling using Google, sklearn and other library documentation, Stack Overflow, AI tools, or other avenues. 

Some suggestions to get you started:
- Look up the assumptions of *linear* regression. Would our dataset violate any of these assumptions?
- What about multiple linear regression? 
- Investigate the performance of your model on different subsets of the data. Are the errors lower for male and female patients? How about across the different facilities? 

In [ ]:
# YOUR CODE HERE